In [ ]:
# Please install OpenAI SDK first: `pip3 install openai`

from openai import OpenAI

client = OpenAI(api_key="", base_url="https://api.deepseek.com")#Your model API key here

messages=[
      {"role": "user", "content":
       """

You are a helpful assistant. Please solve this puzzle for me with the help of an external scratchpad. I will provide a task description, guidelines on using the scratchpad, and examples on how to use the scratchpad to solve your puzzle.
Task Description:
There are three pegs and n disks of different sizes stacked on the first peg.
The disks are numbered from 1 (smallest) to n (largest). Disk moves in this puzzle should follow:
1. Only one disk can be moved at a time.
2. Each move consists of taking the upper disk from one stack and placing it on top of another stack.
3. A larger disk may not be placed on top of a smaller disk.
The goal is to move the entire stack to the third peg.
Example: With 3 disks numbered 1 (smallest), 2, and 3 (largest), the initial state is [[3, 2, 1], [], []], and a solution might be:
moves = [[1 , 0 , 2] , [2 , 0 , 1] , [1 , 2 , 1] , [3 , 0 , 2] , [1 , 1 , 0] , [2 , 1 , 2] , [1 , 0 , 2]]
This means: Move disk 1 from peg 0 to peg 2, then move disk 2 from peg 0 to peg 1, and so on. Requirements:
• When exploring potential solutions in your thinking process, always include the corresponding complete list of moves.
• The positions are 0-indexed (the leftmost peg is 0).
• Ensure your final answer includes the complete list of moves in the format: moves = [[disk id, from peg, to peg], ...]

I have a puzzle with 9 disks of different sizes with Initial configuration:
• Peg 0: 9 (bottom), . . . 2, 1 (top)
• Peg 1: (empty)
• Peg 2: (empty)
Goal configuration:
• Peg 0: (empty)
• Peg 1: (empty)
• Peg 2: 9 (bottom), . . . 2, 1 (top)
Rules:
• Only one disk can be moved at a time.
• Only the top disk from any stack can be moved.
• A larger disk may not be placed on top of a smaller disk.
Find the sequence of moves to transform the initial configuration into the goal configuration.

Scratchpad Description: When solving the puzzle, the output solution may be extremely long. The real puzzle you will receive may involve many more objects, so a complete solution can contain **hundreds of moves**. Thus, you can choose when to pause your output and save your current state to a scratchpad. In your next round of conversation, you can use the scratchpad to recover your memory of the current round.

- Scratchpad budget: You may write to the scratchpad at most **5 times total** in the whole dialogue. A counter will be provided in each user message. If you exceed the budget, all your output will be ignored. No matter how many times we use the scratchpad in our examples, you can feel free to use it 0 to 5 times based on your own decision.

- Scratchpad interface:
Input: In each round of conversation, I will give you the content in the previous scratchpad at the end of the prompt in JSON format. My input interface is:

{
  "prev_scratchpad": "...",
  "use_count": 0 | 1 | 2 | 3 | …,
  "max_use": 5
}

Output: In each round of conversation, you must follow **this exact interface** on every turn (no extra keys, no prose outside JSON) to answer the puzzle and use the scratchpad.

If you want to use the scratchpad and start a new round, you can return:

{
  "moves": "...",    # your partial answer
  "scratchpad" : "...",                # optional private notes for THIS turn
  "finished"   : "False"      # "True" only when the task is fully solved
}

If you think the task is fully solved, you can return:

{
  "moves" : "...",    # your partial answer of the final round
  "scratchpad" : "",                # empty
  "finished"   : "True"      # "True" only when the task is fully solved
}

After this, we will collect all the moves in all rounds and evaluate your answer. The content in the scratchpad will not be evaluated.

Scratchpad Examples:
[Example 1]
Task description: With 3 disks numbered 1 (smallest), 2, and 3 (largest), the initial state is [[3, 2, 1], [], []]
Example description: Solve this task in one step

Round 1
Input: {
  "prev_scratchpad": "",
  "use_count": 0,
  "max_use": 5
}

Output: {
  "moves": "[[1 , 0 , 2] , [2 , 0 , 1] , [1 , 2 , 1] , [3 , 0 , 2] , [1 , 1 , 0] , [2 , 1 , 2] , [1 , 0 , 2]]",
  "scratchpad": "",
  "finished": "True"
}

[Example 2]
Task description: With 3 disks numbered 1 (smallest), 2, and 3 (largest), the initial state is [[3, 2, 1], [], []]
Example description: Solve this task in two steps

Round 1
Input: {
  "prev_scratchpad": "",
  "use_count": 0,
  "max_use": 5
}

Output: {
  "moves" : "[[1 , 0 , 2] , [2 , 0 , 1]",
  "scratchpad": "state = [[3], [2], [1]]",
  "finished": "False"
}

Round 2
Input: {
  "prev_scratchpad": "state = [[3], [2], [1]]",
  "use_count": 1,
  "max_use": 5
}

Output: {
  "moves" : "[[1 , 2 , 1] , [3 , 0 , 2] , [1 , 1 , 0] , [2 , 1 , 2] , [1 , 0 , 2]]",
  "scratchpad": "",
  "finished": "True"
}


[Example 3]
Task description: With 3 disks numbered 1 (smallest), 2, and 3 (largest), the initial state is [[3, 2, 1], [], []]
Example description: Solve this task in three steps
Round 1
Input: {
  "prev_scratchpad": "",
  "use_count": 0,
  "max_use": 5
}

Output: {
  "moves" : "[[1 , 0 , 2] , [2 , 0 , 1]",
  "scratchpad": "state = [[3], [2], [1]]",
  "finished": "False"
}

Round 2
Input: {
  "prev_scratchpad": "state = [[3], [2], [1]]",
  "use_count": 1,
  "max_use": 5
}

Output: {
  "moves" : "[[1 , 2 , 1] , [3 , 0 , 2]]",
  "scratchpad": "state = [[], [2 , 1], [3]]",
  "finished": "True"
}

Round 3
Input: {
  "prev_scratchpad": "state = [[], [2 , 1], [3]]",
  "use_count": 2,
  "max_use": 5
}

Output: {
  "moves" : "[[1 , 1 , 0] , [2 , 1 , 2] , [1 , 0 , 2]]",
  "scratchpad": "[]",
  "finished": "True"
}


Previous Scratchpad:
{
  "prev_scratchpad": "",
  "use_count": 0,
  "max_use": 5
}


"""},
    ]

response = client.chat.completions.create(
    model="deepseek-chat",
    max_tokens=8192,
    timeout= 1200,
    messages=messages,
    stream=False
)

print(response.choices[0].message.content)

The Tower of Hanoi problem with 9 disks requires a sequence of 2^9 - 1 = 511 moves. Given the constraints, I'll break this down into manageable chunks using the scratchpad to keep track of the state.

### Initial State:
- Peg 0: [9, 8, 7, 6, 5, 4, 3, 2, 1]
- Peg 1: []
- Peg 2: []

### Goal State:
- Peg 0: []
- Peg 1: []
- Peg 2: [9, 8, 7, 6, 5, 4, 3, 2, 1]

### Approach:
The recursive solution involves:
1. Moving the top n-1 disks from the source peg to the auxiliary peg.
2. Moving the nth disk from the source peg to the target peg.
3. Moving the n-1 disks from the auxiliary peg to the target peg.

For 9 disks, this will be broken down into smaller steps.

### First Chunk of Moves (Moving disks 1-8 from peg 0 to peg 1):
This involves moving the top 8 disks from peg 0 to peg 1, using peg 2 as the auxiliary.

Here's the first set of moves:

```json
{
  "moves": "[[1, 0, 2], [2, 0, 1], [1, 2, 1], [3, 0, 2], [1, 1, 0], [2, 1, 2], [1, 0, 2], [4, 0, 1], [1, 2, 1], [2, 2, 0], [1, 1, 0], [3, 2

In [ ]:
import json
import re

def extract_scratchpad(llm_content: str) -> str | None:
    """
    Return the value of the "scratchpad" field from an LLM reply.

    Parameters
    ----------
    llm_content : str
        The raw string you get from response.choices[0].message.content.

    Returns
    -------
    str | None
        The scratchpad text if it exists, otherwise None.
    """
    # 1️⃣  Try the clean-way first: assume it’s valid JSON
    try:
        data = json.loads(llm_content)
        return data.get("scratchpad")
    except Exception:
        pass  # fall through if json.loads() fails

    # 2️⃣  Fallback: use a regex to grab the value in quotes
    m = re.search(r'"scratchpad"\s*:\s*"([^"]*)"', llm_content)
    if m:
        # Unescape any \" sequences that were inside the JSON-ish string
        return bytes(m.group(1), "utf-8").decode("unicode_escape")

    return None


# ------- usage example -------
content = response.choices[0].message.content  # your raw LLM output
scratchpad_text = extract_scratchpad(content)
print("Scratchpad:", scratchpad_text)


Scratchpad: state = [[9], [8, 7, 6, 5, 4, 3, 2, 1], []]


In [ ]:
scratchpad=scratchpad_text

messages=[
      {"role": "user", "content":
       f"""

You are a helpful assistant. Please solve this puzzle for me with the help of an external scratchpad. I will provide a task description, guidelines on using the scratchpad, and examples on how to use the scratchpad to solve your puzzle.
Task Description:
There are three pegs and n disks of different sizes stacked on the first peg.
The disks are numbered from 1 (smallest) to n (largest). Disk moves in this puzzle should follow:
1. Only one disk can be moved at a time.
2. Each move consists of taking the upper disk from one stack and placing it on top of another stack.
3. A larger disk may not be placed on top of a smaller disk.
The goal is to move the entire stack to the third peg.
Example: With 3 disks numbered 1 (smallest), 2, and 3 (largest), the initial state is [[3, 2, 1], [], []], and a solution might be:
moves = [[1 , 0 , 2] , [2 , 0 , 1] , [1 , 2 , 1] , [3 , 0 , 2] , [1 , 1 , 0] , [2 , 1 , 2] , [1 , 0 , 2]]
This means: Move disk 1 from peg 0 to peg 2, then move disk 2 from peg 0 to peg 1, and so on. Requirements:
• When exploring potential solutions in your thinking process, always include the corresponding complete list of moves.
• The positions are 0-indexed (the leftmost peg is 0).
• Ensure your final answer includes the complete list of moves in the format: moves = [[disk id, from peg, to peg], ...]

I have a puzzle with 9 disks of different sizes with Initial configuration:
• Peg 0: 9 (bottom), . . . 2, 1 (top)
• Peg 1: (empty)
• Peg 2: (empty)
Goal configuration:
• Peg 0: (empty)
• Peg 1: (empty)
• Peg 2: 9 (bottom), . . . 2, 1 (top)
Rules:
• Only one disk can be moved at a time.
• Only the top disk from any stack can be moved.
• A larger disk may not be placed on top of a smaller disk.
Find the sequence of moves to transform the initial configuration into the goal configuration.

Scratchpad Description: When solving the puzzle, the output solution may be extremely long. The real puzzle you will receive may involve many more objects, so a complete solution can contain **hundreds of moves**. Thus, you can choose when to pause your output and save your current state to a scratchpad. In your next round of conversation, you can use the scratchpad to recover your memory of the current round.

- Scratchpad budget: You may write to the scratchpad at most **5 times total** in the whole dialogue. A counter will be provided in each user message. If you exceed the budget, all your output will be ignored. No matter how many times we use the scratchpad in our examples, you can feel free to use it 0 to 5 times based on your own decision.

- Scratchpad interface:
Input: In each round of conversation, I will give you the content in the previous scratchpad at the end of the prompt in JSON format. My input interface is:

{{
  "prev_scratchpad": "...",
  "use_count": 0 | 1 | 2 | 3 | …,
  "max_use": 5
}}

Output: In each round of conversation, you must follow **this exact interface** on every turn (no extra keys, no prose outside JSON) to answer the puzzle and use the scratchpad.

If you want to use the scratchpad and start a new round, you can return:

{{
  "moves": "...”",    # your partial answer
  "scratchpad" : "...",                # optional private notes for THIS turn
  "finished"   : "False"      # "True" only when the task is fully solved
}}

If you think the task is fully solved, you can return:

{{
  "moves": "...",    # your partial answer of the final round
  "scratchpad" : "",                # empty
  "finished"   :"True"      # "True" only when the task is fully solved
}}

After this, we will collect all the moves in all rounds and evaluate your answer. The content in the scratchpad will not be evaluated.

Scratchpad Examples:
[Example 1]
Task description: With 3 disks numbered 1 (smallest), 2, and 3 (largest), the initial state is [[3, 2, 1], [], []]
Example description: Solve this task in one step

Round 1
Input: {{
  "prev_scratchpad": "",
  "use_count": 0,
  "max_use": 5
}}

Output: {{
  "moves": "[[1 , 0 , 2] , [2 , 0 , 1] , [1 , 2 , 1] , [3 , 0 , 2] , [1 , 1 , 0] , [2 , 1 , 2] , [1 , 0 , 2]]",
  "scratchpad": "",
  "finished": "True"
}}

[Example 2]
Task description: With 3 disks numbered 1 (smallest), 2, and 3 (largest), the initial state is [[3, 2, 1], [], []]
Example description: Solve this task in two steps

Round 1
Input: {{
  "prev_scratchpad": "",
  "use_count": 0,
  "max_use": 5
}}

Output: {{
  "moves" : "[[1 , 0 , 2] , [2 , 0 , 1]",
  "scratchpad": "state = [[3], [2], [1]]",
  "finished": "False"
}}

Round 2
Input: {{
  "prev_scratchpad": "state = [[3], [2], [1]]",
  "use_count": 1,
  "max_use": 5
}}

Output: {{
  "moves" : "[[1 , 2 , 1] , [3 , 0 , 2] , [1 , 1 , 0] , [2 , 1 , 2] , [1 , 0 , 2]]",
  "scratchpad": "",
  "finished": "True"
}}


[Example 3]
Task description: With 3 disks numbered 1 (smallest), 2, and 3 (largest), the initial state is [[3, 2, 1], [], []]
Example description: Solve this task in three steps
Round 1
Input: {{
  "prev_scratchpad": "",
  "use_count": 0,
  "max_use": 5
}}

Output: {{
  "moves" : "[[1 , 0 , 2] , [2 , 0 , 1]",
  "scratchpad": "state = [[3], [2], [1]]",
  "finished": "False"
}}

Round 2
Input: {{
  "prev_scratchpad": "state = [[3], [2], [1]]",
  "use_count": 1,
  "max_use": 5
}}

Output: {{
  "moves" : "[[1 , 2 , 1] , [3 , 0 , 2]]",
  "scratchpad": "state = [[], [2 , 1], [3]]",
  "finished": "True"
}}

Round 3
Input: {{
  "prev_scratchpad": "state = [[], [2 , 1], [3]]",
  "use_count": 2,
  "max_use": 5
}}

Output: {{
  "moves" : "[[1 , 1 , 0] , [2 , 1 , 2] , [1 , 0 , 2]]",
  "scratchpad": "[]",
  "finished": "True"
}}


Previous Scratchpad:
{{
  "prev_scratchpad": "{scratchpad}",
  "use_count": 1,
  "max_use": 5
}}


"""},
]

response = client.chat.completions.create(
    model="deepseek-reasoner",
    max_tokens=8192,
    timeout= 1200,
    messages=messages,
    stream=False
)

print(response.choices[0].message.content)

In [ ]:
from typing import List, Tuple

# ---------- 类型别名 ----------
Move = Tuple[int, int, int]   # (disk, src_peg, dst_peg)  ── 0·1·2 柱编号

# ---------- 校验器 ----------
def validate_hanoi_moves(moves: List[Move],
                         n_disks: int | None = None,     # ← 可以省略
                         use_one_based: bool = False,
                         verbose: bool = False) -> bool:

    if n_disks is None:
      try:
          n_disks = max(disk for disk, _, _ in moves)
      except ValueError:          # moves 为空
          raise ValueError("moves list is empty; cannot infer disk count")
    """
    检验动作序列能否把 N 盘汉诺塔从柱 0(或1) 完整合法地移动到柱 2(或3)。
    """
    # 选择下标映射
    if use_one_based:
        def idx(i: int) -> int:
            if i not in {1, 2, 3}:
                raise ValueError(f"Peg must be 1,2,3; got {i}")
            return i - 1
    else:
        def idx(i: int) -> int:
            if i not in {0, 1, 2}:
                raise ValueError(f"Peg must be 0,1,2; got {i}")
            return i

    # --- 构造初始 / 目标 ---
    pegs = [list(range(n_disks, 0, -1)), [], []]  # 初始：全部在柱 0
    goal = [[], [], list(range(n_disks, 0, -1))]  # 目标：全部在柱 2

    def top(p):                  # 取柱顶盘号；空柱返回 None
        return pegs[p][-1] if pegs[p] else None

    # --- 逐步检验 ---
    for step, (disk, src, dst) in enumerate(moves, start=1):
        s, d = idx(src), idx(dst)

        # 1) 盘号合法
        if not (1 <= disk <= n_disks):
            raise ValueError(f"Step {step}: disk {disk} out of range 1…{n_disks}")

        # 2) 源柱非空 & 顶端必须正是该盘
        if top(s) != disk:
            raise ValueError(
                f"Step {step}: top of peg {src} is {top(s)}, expected {disk}")

        # 3) 目标柱合法（小盘不能压在大盘下）
        if top(d) is not None and top(d) < disk:
            raise ValueError(
                f"Step {step}: cannot place disk {disk} on smaller disk {top(d)}")

        # 4) 执行移动
        pegs[s].pop()
        pegs[d].append(disk)

        if verbose:
            print(f"After step {step}: {pegs}")

    # --- 最终状态检查 ---
    if pegs != goal:
        raise AssertionError(
            f"Final state differs from goal.\nGot : {pegs}\nGoal: {goal}")

    if verbose:
        print("✓ All moves valid, goal reached!")
    return True


# ---------- 使用示例 ----------
if __name__ == "__main__":
    moves = [

        [1, 0, 2], [2, 0, 1], [1, 2, 1], [3, 0, 2], [1, 1, 0], [2, 1, 2], [1, 0, 2], [4, 0, 1], [1, 2, 1], [2, 2, 0], [1, 1, 0], [3, 2, 1], [1, 0, 2], [2, 0, 1], [1, 2, 1], [5, 0, 2], [1, 1, 0], [2, 1, 2], [1, 0, 2], [3, 1, 0], [1, 2, 1], [2, 2, 0], [1, 1, 0], [4, 1, 2], [1, 0, 2], [2, 0, 1], [1, 2, 1], [3, 0, 2], [1, 1, 0], [2, 1, 2], [1, 0, 2], [6, 0, 1], [1, 2, 1], [2, 2, 0], [1, 1, 0], [3, 2, 1], [1, 0, 2], [2, 0, 1], [1, 2, 1], [4, 2, 0], [1, 1, 0], [2, 1, 2], [1, 0, 2], [3, 1, 0], [1, 2, 1], [2, 2, 0], [1, 1, 0], [5, 2, 1], [1, 0, 2], [2, 0, 1], [1, 2, 1], [3, 0, 2], [1, 1, 0], [2, 1, 2], [1, 0, 2], [4, 0, 1], [1, 2, 1], [2, 2, 0], [1, 1, 0], [3, 2, 1], [1, 0, 2], [2, 0, 1], [1, 2, 1],
        [7, 0, 2], [1, 1, 0], [2, 1, 2], [1, 0, 2], [3, 1, 0], [1, 2, 1], [2, 2, 0], [1, 1, 0], [4, 1, 2], [1, 0, 2], [2, 0, 1], [1, 2, 1], [3, 0, 2], [1, 1, 0], [2, 1, 2], [1, 0, 2], [5, 1, 0], [1, 2, 1], [2, 2, 0], [1, 1, 0], [3, 2, 1], [1, 0, 2], [2, 0, 1], [1, 2, 1], [4, 2, 0], [1, 1, 0], [2, 1, 2], [1, 0, 2], [3, 1, 0], [1, 2, 1], [2, 2, 0], [1, 1, 0], [6, 1, 2], [1, 0, 2], [2, 0, 1], [1, 2, 1], [3, 0, 2], [1, 1, 0], [2, 1, 2], [1, 0, 2], [4, 0, 1], [1, 2, 1], [2, 2, 0], [1, 1, 0], [3, 2, 1], [1, 0, 2], [2, 0, 1], [1, 2, 1], [5, 0, 2], [1, 1, 0], [2, 1, 2], [1, 0, 2], [3, 1, 0], [1, 2, 1], [2, 2, 0], [1, 1, 0], [4, 1, 2], [1, 0, 2], [2, 0, 1], [1, 2, 1], [3, 0, 2], [1, 1, 0], [2, 1, 2], [1, 0, 2]

        ]

    validate_hanoi_moves(moves, verbose=True)  # ← 1-3 编号示例

    # 如果你的 moves 用 0-2 下标：
    # validate_hanoi_moves(moves_zero_based, n_disks=N, use_one_based=False, verbose=True)

After step 1: [[7, 6, 5, 4, 3, 2], [], [1]]
After step 2: [[7, 6, 5, 4, 3], [2], [1]]
After step 3: [[7, 6, 5, 4, 3], [2, 1], []]
After step 4: [[7, 6, 5, 4], [2, 1], [3]]
After step 5: [[7, 6, 5, 4, 1], [2], [3]]
After step 6: [[7, 6, 5, 4, 1], [], [3, 2]]
After step 7: [[7, 6, 5, 4], [], [3, 2, 1]]
After step 8: [[7, 6, 5], [4], [3, 2, 1]]
After step 9: [[7, 6, 5], [4, 1], [3, 2]]
After step 10: [[7, 6, 5, 2], [4, 1], [3]]
After step 11: [[7, 6, 5, 2, 1], [4], [3]]
After step 12: [[7, 6, 5, 2, 1], [4, 3], []]
After step 13: [[7, 6, 5, 2], [4, 3], [1]]
After step 14: [[7, 6, 5], [4, 3, 2], [1]]
After step 15: [[7, 6, 5], [4, 3, 2, 1], []]
After step 16: [[7, 6], [4, 3, 2, 1], [5]]
After step 17: [[7, 6, 1], [4, 3, 2], [5]]
After step 18: [[7, 6, 1], [4, 3], [5, 2]]
After step 19: [[7, 6], [4, 3], [5, 2, 1]]
After step 20: [[7, 6, 3], [4], [5, 2, 1]]
After step 21: [[7, 6, 3], [4, 1], [5, 2]]
After step 22: [[7, 6, 3, 2], [4, 1], [5]]
After step 23: [[7, 6, 3, 2, 1], [4], [5]]
After st